# Classical Pre-processing

In [1]:
import qiskit_machine_learning
qiskit_machine_learning.__version__

'0.7.2'

In [2]:
import numpy as np # type: ignore
import tensorflow as tf # type: ignore
from tensorflow.keras import datasets, layers, models   # type: ignore
from tensorflow.keras.utils import to_categorical # type: ignore
import time
import matplotlib.pyplot as plt # type: ignore
import warnings # type: ignore
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler, StandardScaler

from qiskit.circuit.library import ZZFeatureMap
from qiskit.primitives import Sampler

from qiskit_machine_learning.algorithms import QSVC
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit_algorithms.state_fidelities import ComputeUncompute

import ssl
ssl._create_default_https_context = ssl._create_unverified_context
 
import os
import certifi # type: ignore
os.environ['SSL_CERT_FILE'] = certifi.where()

In [3]:
# Load the CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()

# Randomly select 2 classes
num_classes = 10
selected_classes = np.random.choice(range(num_classes), 2, replace=False)
print(f"Selected classes: {selected_classes}")

# Filter the train and test datasets to include only the selected classes
train_mask = np.isin(y_train, selected_classes)
test_mask = np.isin(y_test, selected_classes)

x_train = x_train[train_mask.squeeze()]
y_train = y_train[train_mask.squeeze()]
x_test = x_test[test_mask.squeeze()]
y_test = y_test[test_mask.squeeze()]

idx_train = 15
idx_test = 5
x_train = x_train[:idx_train]
x_test = x_test[:idx_test]
y_train = y_train[:idx_train]
y_test = y_test[:idx_test]

# Resize images to 8x8
# Resize images to 4x4

im_size = 4
x_train = tf.image.resize(x_train, (im_size, im_size)).numpy()
x_test = tf.image.resize(x_test, (im_size, im_size)).numpy()

# Normalize the pixel values to [0, 1]
x_train, x_test = x_train / 255.0, x_test / 255.0

# One-hot encode the labels for the selected classes
y_train = to_categorical(y_train, num_classes=num_classes)[:, selected_classes]
y_test = to_categorical(y_test, num_classes=num_classes)[:, selected_classes]

# Display the shapes of the processed datasets
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)


Selected classes: [8 3]
(15, 4, 4, 3) (5, 4, 4, 3) (15, 2) (5, 2)


In [4]:
x_train_gray = tf.image.rgb_to_grayscale(x_train).numpy()
x_test_gray = tf.image.rgb_to_grayscale(x_test).numpy()

# Reshape the data to 1D vectors
x_train_rs = x_train_gray.reshape(x_train_gray.shape[0], -1)
x_test_rs = x_test_gray.reshape(x_test_gray.shape[0], -1)

from sklearn.preprocessing import StandardScaler, MinMaxScaler
# Standardize the data to have a mean of 0 and variance of 1
scaler = StandardScaler()
x_train_s = scaler.fit_transform(x_train_rs)
x_test_s = scaler.transform(x_test_rs)

x_train_s.shape , x_test_s.shape

((15, 16), (5, 16))

In [5]:
# Apply PCA for feature reduction
from sklearn.decomposition import PCA
n_components = 8
pca = PCA(n_components=n_components)
x_train_pca = pca.fit_transform(x_train_s)
x_test_pca = pca.transform(x_test_s)

# Output the shape of the reduced data
print(f"Original shape: {x_train_s.shape}")
print(f"Reduced shape: {x_train_pca.shape}")

# Optional: Show explained variance to understand the amount of information retained
explained_variance = np.sum(pca.explained_variance_ratio_)
print(f"Explained variance by {n_components} components: {explained_variance * 100:.2f}%")

Original shape: (15, 16)
Reduced shape: (15, 8)
Explained variance by 8 components: 95.08%


# Quantum SVM with feature map

In [6]:
import time
%time
t0 = time.time()

feature_map = ZZFeatureMap(feature_dimension = n_components, entanglement= "full")

fidelity = ComputeUncompute(sampler=Sampler())
new_kernel = FidelityQuantumKernel(feature_map=feature_map, fidelity=fidelity)

qsvc = QSVC(quantum_kernel=new_kernel)

qsvc.fit(x_train_pca, y_train.argmax(axis=1))
t1 = time.time()
print(f'Training set accuracy {qsvc.score(x_train_pca, y_train.argmax(axis=1))}')


CPU times: user 1 µs, sys: 1e+03 ns, total: 2 µs
Wall time: 3.1 µs
Training set accuracy 1.0


In [7]:
print(f"Test set accuracy: {qsvc.score(x_test_pca, y_test.argmax(axis=1))}")
t2 = time.time()

Test set accuracy: 0.4


In [8]:
print(f"Time taken for training and testing:{t1-t0, t2-t1}")

Time taken for training and testing:(0.9252398014068604, 1.7010149955749512)


# QSVM Insights: 

1. Training set accuracy 100%
2. Testing set accuracy with SVM: 50.00%
3. Train dataset size = 500, test size = 10
4. Randomly chosen 2 classes out of 10
5. Eache image is 8x8 pizel in size
6. Train time: 22 s, Test time: 25 s

# QSVM - toy dataset Insights: 

1. Training set accuracy 100%
2. Testing set accuracy with SVM: 40.00%
3. Train dataset size = 15, test size = 5
4. Randomly chosen 2 classes out of 10
5. Eache image is 4x4 pizel in size
6. Train time: 1.03 s, Test time: 5.36 s